In [2]:
import torch

In [3]:
heat_map = torch.rand([128,128,80])
pred_wh = torch.rand([128,128,2])
pred_offset = torch.rand([128,128,2])

In [4]:
heat_map

tensor([[[0.7010, 0.8813, 0.3542,  ..., 0.1363, 0.8566, 0.9529],
         [0.1124, 0.4454, 0.6705,  ..., 0.9078, 0.5223, 0.2276],
         [0.7630, 0.8487, 0.7672,  ..., 0.1675, 0.7345, 0.7258],
         ...,
         [0.9320, 0.4645, 0.9271,  ..., 0.8326, 0.9778, 0.9578],
         [0.1572, 0.0357, 0.0141,  ..., 0.0422, 0.7889, 0.4760],
         [0.5205, 0.1396, 0.7239,  ..., 0.0925, 0.3858, 0.5362]],

        [[0.0459, 0.8894, 0.8997,  ..., 0.0166, 0.8605, 0.5895],
         [0.9171, 0.3693, 0.0312,  ..., 0.2074, 0.0296, 0.0189],
         [0.9781, 0.9802, 0.1552,  ..., 0.2169, 0.0495, 0.8259],
         ...,
         [0.4568, 0.8767, 0.4681,  ..., 0.4417, 0.7408, 0.7504],
         [0.5906, 0.1073, 0.0170,  ..., 0.4430, 0.7480, 0.5811],
         [0.1789, 0.6684, 0.1726,  ..., 0.9524, 0.9379, 0.7664]],

        [[0.9093, 0.6771, 0.4679,  ..., 0.8713, 0.0487, 0.0336],
         [0.2850, 0.9286, 0.7278,  ..., 0.6076, 0.8065, 0.4010],
         [0.4243, 0.7959, 0.7941,  ..., 0.9590, 0.8239, 0.

In [5]:
yv, xv = torch.meshgrid(torch.arange(0, 128), torch.arange(0,128))

/home/sjtu/anaconda3/envs/monai_zt/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
# xv = xv.flatten().float()
# yv = yv.flatten().float()

In [7]:
torch.max(heat_map,dim=-1)

torch.return_types.max(
values=tensor([[0.9633, 0.9984, 0.9964,  ..., 0.9921, 0.9911, 0.9924],
        [0.9830, 0.9978, 0.9996,  ..., 0.9888, 0.9989, 0.9921],
        [0.9835, 0.9947, 0.9932,  ..., 0.9930, 0.9911, 0.9868],
        ...,
        [0.9968, 0.9906, 0.9981,  ..., 0.9950, 0.9966, 0.9962],
        [0.9820, 0.9785, 0.9971,  ..., 0.9987, 0.9912, 0.9778],
        [0.9703, 0.9976, 0.9796,  ..., 0.9992, 0.9975, 0.9867]]),
indices=tensor([[28, 73, 29,  ..., 56, 32, 31],
        [46, 56, 76,  ..., 44, 14, 30],
        [ 4, 24, 57,  ...,  7, 25,  2],
        ...,
        [ 1, 61, 68,  ..., 33, 19, 74],
        [66,  6,  3,  ..., 10,  8, 28],
        [18, 42, 27,  ..., 16, 53, 63]]))

In [8]:
class_conf, class_pred = torch.max(heat_map,dim=-1)

In [9]:
mask = class_conf > 0.995

In [10]:
pred_wh_mask = pred_wh[mask]
pred_offset_mask = pred_offset[mask]

In [11]:
xv_mask = torch.unsqueeze(xv[mask] + pred_offset_mask[..., 0],-1)
yv_mask = torch.unsqueeze(yv[mask] + pred_offset_mask[..., 1],-1)

In [12]:
half_w, half_h = pred_wh_mask[...,0] / 2, pred_wh_mask[...,1] / 2
bboxes = torch.cat([xv_mask - half_w, yv_mask - half_h, xv_mask + half_w, yv_mask + half_h], dim=1)

In [14]:
xv_mask

tensor([[  1.8619],
        [  2.5618],
        [  6.9185],
        ...,
        [122.1024],
        [125.0149],
        [126.2360]])

In [15]:
half_w

tensor([0.4643, 0.3799, 0.0951,  ..., 0.2329, 0.1506, 0.2350])

In [16]:
xv_mask - half_w

tensor([[  1.3976,   1.4820,   1.7668,  ...,   1.6290,   1.7113,   1.6269],
        [  2.0976,   2.1819,   2.4668,  ...,   2.3289,   2.4113,   2.3269],
        [  6.4542,   6.5385,   6.8234,  ...,   6.6856,   6.7679,   6.6835],
        ...,
        [121.6381, 121.7225, 122.0073,  ..., 121.8695, 121.9518, 121.8674],
        [124.5506, 124.6350, 124.9199,  ..., 124.7820, 124.8643, 124.7800],
        [125.7717, 125.8561, 126.1409,  ..., 126.0031, 126.0854, 126.0010]])

In [18]:
bboxes[:,[0,2]] /= 128

In [19]:
bboxes[:,[0,2]]

tensor([[0.0109, 0.0138],
        [0.0164, 0.0193],
        [0.0504, 0.0533],
        ...,
        [0.9503, 0.9532],
        [0.9731, 0.9759],
        [0.9826, 0.9855]])

实验二，e2ec train_decode.py

In [22]:
import torch

In [23]:
wh_pred = torch.tensor([16,256,128,128])
ct_01 = torch.randn([16,23]).bool()
batch = {'ct_ind' : 128 * torch.rand([16,23])}

In [24]:
len(wh_pred)

4

In [57]:
batch['ct_ind'].shape

torch.Size([16, 23])

In [58]:
ct_ind = batch['ct_ind'][ct_01]

In [59]:
ct_ind

tensor([8.5172e+01, 1.8720e+01, 9.4150e+01, 1.0494e+02, 6.4537e+01, 7.1927e+01,
        5.5795e+01, 3.8750e+01, 1.1634e+02, 5.5276e+01, 8.0368e+01, 1.1895e+02,
        9.9145e+01, 3.5546e+01, 5.5215e+01, 6.1898e+01, 4.7944e+01, 9.3357e+01,
        1.1757e+00, 1.1716e+01, 1.1762e+01, 1.1301e+01, 6.2224e+01, 1.7206e+01,
        3.0724e+01, 5.2435e-01, 9.4425e+01, 3.6369e+01, 8.6609e+01, 3.9085e+01,
        3.7400e+01, 4.3559e+01, 4.5815e+01, 1.6852e+01, 7.4340e+01, 7.2415e+01,
        1.2699e+02, 6.2914e+01, 2.7151e+01, 1.5683e+01, 2.7345e+01, 4.3131e+01,
        1.0762e+02, 6.7313e+01, 6.7979e+01, 6.9851e+01, 1.8232e+01, 5.5839e+01,
        6.8987e+01, 1.2267e+02, 9.6060e+01, 5.6023e+01, 6.9457e+01, 2.4932e+01,
        6.5889e+01, 1.2063e+00, 1.1126e+02, 1.2414e+02, 1.0446e+02, 3.9996e+01,
        2.1316e+01, 4.6455e+01, 5.9422e+00, 1.1235e+02, 4.6997e+01, 1.2502e+02,
        8.9761e+01, 1.1835e+02, 5.3900e+00, 4.6601e+01, 8.6437e+00, 4.2416e+01,
        4.5221e+01, 1.2362e+02, 1.0387e+

In [2]:
def train_decode(self, data_input, output, cnn_feature):  # data_input=batch, cnn_feature=cnn_feature, ouput=output
        wh_pred = output['wh'] #torch.Size([16, 256, 128, 128])  coco 128*2
        ct_01 = data_input['ct_01'].bool()
        ct_ind = data_input['ct_ind'][ct_01]
        ct_img_idx = data_input['ct_img_idx'][ct_01]
        _, _, height, width = data_input['ct_hm'].size() #128,128
        ct_x, ct_y = ct_ind % width, ct_ind // width #猜一猜ct_ind是什么:-)

        if ct_x.size(0) == 0:
            ct_offset = wh_pred[ct_img_idx, :, ct_y, ct_x].view(ct_x.size(0), 1, 2)
        else:
            ct_offset = wh_pred[ct_img_idx, :, ct_y, ct_x].view(ct_x.size(0), -1, 2)

        ct_x, ct_y = ct_x[:, None].to(torch.float32), ct_y[:, None].to(torch.float32)
        ct = torch.cat([ct_x, ct_y], dim=1)

        init_polys = ct_offset * self.stride + ct.unsqueeze(1).expand(ct_offset.size(0),
                                                                      ct_offset.size(1), ct_offset.size(2)) 
        coarse_polys = self.refine(cnn_feature, ct, init_polys, ct_img_idx.clone()) # global deformation

        output.update({'poly_init': init_polys * self.down_sample})
        output.update({'poly_coarse': coarse_polys * self.down_sample})
        return

实验三：ct

In [43]:
import torch
import warnings
warnings.filterwarnings('ignore')

In [44]:
ct_01 = torch.randn([16,23])
ct_01 = ct_01.ge(0)  
wh_pred = torch.rand([16,256,128,128])
width = 128

In [45]:
ct_01.shape

torch.Size([16, 23])

In [46]:
ct_ind = torch.rand([16,23])
ct_img_idx = torch.rand([16,23])

In [47]:
ct_ind = ct_ind[ct_01]
ct_img_idx = ct_img_idx[ct_01]

In [ ]:
ct_ind

In [50]:
ct_x, ct_y = ct_ind % width, ct_ind // width

In [37]:
ct_x.shape

torch.Size([173])

In [51]:
# ct_offset = wh_pred[ct_img_idx, :, ct_y, ct_x].view(ct_x.size(0), -1, 2)

IndexError: tensors used as indices must be long, byte or bool tensors

In [66]:
ct_x, ct_y = ct_x[:, None].to(torch.float32), ct_y[:, None].to(torch.float32)

In [ ]:
ct_x

In [69]:
ct = torch.cat([ct_x, ct_y], dim=1)

In [70]:
ct.shape

torch.Size([184, 2])

In [38]:
ct_img_idx = torch.rand([16,23]).bool()
wh_pred = torch.rand([16,256,128,128]).bool()

In [ ]:
ct_offset = wh_pred[ct_img_idx, :, ct_y, ct_x].view(ct_x.size(0), -1, 2)

In [ ]:
ct.unsqueeze(1).expand(ct_offset.size(0),ct_offset.size(1), ct_offset.size(2))